In [1]:
import os
import torch
import gdown
import numpy as np
import scanpy as sc
import pandas as pd
import scarches as sca
import matplotlib.pyplot as plt

 captum (see https://github.com/pytorch/captum).
INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/home/icb/aleksandra.topalova/miniconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/icb/aleksandra.topalova/miniconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
early_stopping_kwargs = {
    "early_stopping_metric": "val_prototype_loss",
    "mode": "min",
    "threshold": 0,
    "patience": 20,
    "reduce_lr": True,
    "lr_patience": 13,
    "lr_factor": 0.1,
}

condition_key = 'study'
cell_type_key = ['cell_type']

In [3]:
# url = 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE211799&format=file&file=GSE211799%5Fadata%5Fatlas%2Eh5ad%2Egz'
# output = 'GSE211799_adata_atlas.h5ad.gz'
# gdown.download(url, output)

In [4]:
# import gzip
# import shutil
# with gzip.open(output, 'rb') as f_in:
#     with open('GSE211799_adata_atlas.h5ad', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [5]:
adata = sc.read('GSE211799_adata_atlas.h5ad')

In [6]:
adata

AnnData object with n_obs × n_vars = 301796 × 31706
    obs: 'study_sample', 'study', 'file', 'reference', 'size_factors_sample', 'phase_cyclone', 's_cyclone', 'g2m_cyclone', 'g1_cyclone', 'sex', 'ins_score', 'ins_high', 'gcg_score', 'gcg_high', 'sst_score', 'sst_high', 'ppy_score', 'ppy_high', 'cell_filtering', 'age', 'strain', 'tissue', 'technique', 'study_sample_design', 'cell_type', 'cell_type_multiplet', 'cell_subtype', 'cell_subtype_multiplet', 'design', 'cell_type_integrated_v1', 'size_factors_integrated', 'pre_cell_type_unified', 'pre_cell_type_original', 'study_parsed', 'cell_type_integrated_v1_parsed', 'cell_type_parsed', 'low_q', 'BETA-DATA_leiden_r1.5', 'BETA-DATA_leiden_r20', 'BETA-DATA_hc_gene_programs', 'BETA-DATA_hc_gene_programs_parsed', 'BETA-DATA_leiden_r1.5_parsed', 'BETA-DATA_leiden_r1.5_parsed_const', 'CXG-DATA_n_genes', 'CXG-DATA_mt_frac', 'CXG-DATA_doublet_score', 'CXG-DATA_log10_n_counts', 'CXG-DATA_age_approxDays', 'CXG-DATA_cell_subtype_immune_reannotatedInte

In [7]:
adata.raw = adata
sc.pp.normalize_total(adata)

In [8]:
scpoli_model = sca.models.scPoli(
    adata=adata,
    condition_key=condition_key,
    cell_type_keys=cell_type_key,
    embedding_dim=3,
)

Embedding dictionary:
 	Num conditions: 9
 	Embedding dim: 3
Encoder Architecture:
	Input Layer in, out and cond: 31706 256 3
	Hidden Layer 1 in/out: 256 64
	Mean/Var Layer in/out: 64 10
Decoder Architecture:
	First Layer in, out and cond:  10 64 3
	Hidden Layer 1 in/out: 64 256
	Output Layer in/out:  256 31706 



In [ ]:
scpoli_model.train(
    n_epochs=50,
    pretraining_epochs=40,
    early_stopping_kwargs=early_stopping_kwargs,
    eta=5,
)

In [ ]:
scpoli_model.save('/scpoli_pancreas', overwrite =True)